In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
dataset = pd.read_csv("netflix_titles_nov_2019.csv")

In [5]:
dataset.head()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,type
0,81193313,Chocolate,NaN,"Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang ...",South Korea,"November 30, 2019",2019,TV-14,1 Season,"International TV Shows, Korean TV Shows, Roman...",Brought together by meaningful meals in the pa...,TV Show
1,81197050,Guatemala: Heart of the Mayan World,"Luis Ara, Ignacio Jaunsolo",Christian Morales,NaN,"November 30, 2019",2019,TV-G,67 min,"Documentaries, International Movies","From Sierra de las Minas to Esquipulas, explor...",Movie
2,81213894,The Zoya Factor,Abhishek Sharma,"Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, ...",India,"November 30, 2019",2019,TV-14,135 min,"Comedies, Dramas, International Movies",A goofy copywriter unwittingly convinces the I...,Movie
3,81082007,Atlantics,Mati Diop,"Mama Sane, Amadou Mbow, Ibrahima Traore, Nicol...","France, Senegal, Belgium","November 29, 2019",2019,TV-14,106 min,"Dramas, Independent Movies, International Movies","Arranged to marry a rich man, young Ada is cru...",Movie
4,80213643,Chip and Potato,NaN,"Abigail Oliver, Andrea Libman, Briana Buckmast...","Canada, United Kingdom",NaN,2019,TV-Y,2 Seasons,Kids' TV,"Lovable pug Chip starts kindergarten, makes ne...",TV Show


In [28]:
def data_details(df):
    
    print("The total movies and shows is :", df.shape[0])
    print("Total numbers of variables is: ", df.shape[1])
    
    print("----------"*10)
    
    print("The dataset columns are listed below: ")
    print(df.columns)
    
    print("----------"*10)
    
    print("The data type of each columns are: ")
    print(df.dtypes)
    
    print("----------"*10)
    print("The Missing rows in each columns are: \n")
    missing_number = df.isnull().sum()
    print(missing_number[missing_number > 0])
    
data_details(dataset)

The total movies and shows is : 5837
Total numbers of variables is:  12
----------------------------------------------------------------------------------------------------
The dataset columns are listed below: 
Index(['show_id', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'type'],
      dtype='object')
----------------------------------------------------------------------------------------------------
The data type of each columns are: 
show_id          int64
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
type            object
dtype: object
----------------------------------------------------------------------------------------------------
The Missing rows in each columns are: 

director      1901
cast           556